In [ ]:
import azureml.core
from azureml.core import Workspace
from azureml.core.experiment import Experiment

ws = Workspace.from_config()
datastore = ws.get_default_datastore()
experiment = Experiment(ws, 'trump-tweets-classification')

In [ ]:
from azureml.core import Environment, Model, Run
from azureml.core.webservice import AksWebservice, LocalWebservice
from azureml.core.compute import AksCompute
from azureml.core.model import InferenceConfig
from azureml.core.conda_dependencies import CondaDependencies

service_name = "trump-tweets-scoring-aml"
model_name = "trump-tweet-classification"

# getting last model
model = next(iter(Model.list(workspace=ws, name=model_name, latest=True)), None,)

environment = Environment("trump-tweet-inferencing")
conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_pip_package("azureml-sdk")
environment.python.conda_dependencies = conda_dep

aks_target = AksCompute(ws, "trump-tweets-inf")

deployment_config = LocalWebservice.deploy_configuration(port=6789)

inference_config = InferenceConfig(entry_script="score.py",
                                   source_directory="./script_folder/04_deploy/",
                                   environment=environment)

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config, overwrite=True)
service.wait_for_deployment(show_output=True)

In [ ]:
import requests

raw_data = "['The media is spreading fake news!']"

input_data = "{\"data\": [" + raw_data + "]}"

headers = {'Content-Type': 'application/json'}

scoring_uri = "http://localhost:6789/score"
resp = requests.post(scoring_uri, input_data, headers=headers)

print("Should be predicted as '2'")
print("prediction:", resp.text)

In [ ]:
import json

request = ["The media is spreading fake news!"]
input_json = json.dumps({"data": request})
predictions = service.run(input_data = input_json)
response = json.loads(predictions)
for item in sorted(response.items(), key=lambda x: x[1], reverse=True):
    k, v = item
    print ("{:<25} {:<4} ".format(k, round(v,2)))

In [ ]:
print(service.get_logs())

In [ ]:
service.delete()